<a href="https://colab.research.google.com/github/Ajay-2007/deep-learning-projects/blob/master/8.variational_autoencoders_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# Create a directory is not exists
sample_dir = 'samples'

if not os.path.exists(sample_dir):
  os.makedirs(sample_dir)

In [0]:
# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [0]:
# MNIST dataset

dataset = torchvision.datasets.MNIST(root='./data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)

# 

In [0]:
# Dataloader
dataloader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

In [0]:
# Variational Autoencoders model

class VAE(nn.Module):
  def __init__(self, image_size=784, h_dim=400, z_dim=20):
    super(VAE, self).__init__()
    self.fc1 = nn.Linear(image_size, h_dim)
    self.fc2 = nn.Linear(h_dim, z_dim)
    self.fc3 = nn.Linear(h_dim, z_dim)
    self.fc4 = nn.Linear(z_dim, h_dim)
    self.fc5 = nn.Linear(h_dim, image_size)
  
  def encode(self, x):
    h = F.relu(self.fc1(x))
    return self.fc2(h), self.fc3(h)
  
  def reparameterize(self, mu, log_var):
    std = torch.exp(log_var/2)
    eps = torch.rand_like(std)
    return mu + eps * std
  
  def decode(self, z):
    h = F.relu(self.fc4(z))
    return F.sigmoid(self.fc5(h))
  
  def forward(self, x):
    mu, log_var = self.encode(x)
    z = self.reparameterize(mu, log_var)
    x_reconst = self.decode(z)
    return x_reconst, mu, log_var


In [0]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
# Start training
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(dataloader):
    # Forward pass
    images = images.to(device).view(-1, image_size)
    x_reconst, mu, log_var = model.forward(images)

    # Compute reconstruction loss and kl divergence
    reconst_loss = F.binary_cross_entropy(x_reconst, images, size_average=False)
    kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    # Backpropagation and optimize
    loss = reconst_loss + kl_div
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print('Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}'.format(
          epoch+1,
          num_epochs,
          i+1,
          len(dataloader),
          reconst_loss.item(),
          kl_div.item() 
      ))

  # performing validation
  with torch.no_grad():
    # Save the sampled images
    z = torch.randn(batch_size, z_dim).to(device)
    out = model.decode(z).view(-1, 1, 28, 28)
    save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

    # Save the reconstructed image
    out, _, _ = model.forward(images)
    x_concat = torch.cat([images.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
    save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/15], Step [100/469], Reconst Loss: 18897.0781, KL Div: 831.1255
Epoch[1/15], Step [200/469], Reconst Loss: 15424.4229, KL Div: 944.5419
Epoch[1/15], Step [300/469], Reconst Loss: 14033.7588, KL Div: 1069.1892
Epoch[1/15], Step [400/469], Reconst Loss: 12817.4062, KL Div: 1115.1172
Epoch[2/15], Step [100/469], Reconst Loss: 12242.3799, KL Div: 1199.4537
Epoch[2/15], Step [200/469], Reconst Loss: 11103.2051, KL Div: 1134.6670
Epoch[2/15], Step [300/469], Reconst Loss: 11413.7461, KL Div: 1212.8529
Epoch[2/15], Step [400/469], Reconst Loss: 11482.8730, KL Div: 1205.9646
Epoch[3/15], Step [100/469], Reconst Loss: 11572.3271, KL Div: 1235.8120
Epoch[3/15], Step [200/469], Reconst Loss: 11105.0576, KL Div: 1230.4738
Epoch[3/15], Step [300/469], Reconst Loss: 10754.8809, KL Div: 1253.6549
Epoch[3/15], Step [400/469], Reconst Loss: 10684.3447, KL Div: 1220.6227
Epoch[4/15], Step [100/469], Reconst Loss: 10405.5312, KL Div: 1237.2039
Epoch[4/15], Step [200/469], Reconst Loss: 10559.1074